In [120]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
import datetime

In [155]:
load_dotenv()
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_TN_FBREF_RESULTS = os.getenv('DB_TN_FBREF_RESULTS')
DB_TN_SOFIFA_TEAMS_STATS = os.getenv('DB_TN_SOFIFA_TEAMS_STATS')
DB_TN_MODELS_RESULTS = os.getenv('DB_TN_MODELS_RESULTS')
MLFLOW_TRACKING_URI = os.getenv('MLFLOW_TRACKING_URI')
DB_TN_ODDS = os.getenv('DB_TN_ODDS')

In [156]:
db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

In [159]:
query_models_results = f"SELECT * FROM {DB_TN_MODELS_RESULTS} WHERE date_match >= :date_match AND model = :model"
query_odds = f"SELECT * FROM {DB_TN_ODDS} WHERE commence_time >= :commence_time"
date_match = '2024-08-26 00:00:00'
model =  'RSF_PR_LR'

In [160]:
date_match = datetime.datetime.strptime(date_match, "%Y-%m-%d %H:%M:%S")
with engine.connect() as connection:
    df_models_results = pd.read_sql(text(query_models_results), connection, params={"date_match": date_match, "model": model})
    df_odds = pd.read_sql(text(query_odds), connection, params={"commence_time": date_match})
    df_team_stats = pd.read_sql(f"SELECT * FROM {DB_TN_SOFIFA_TEAMS_STATS}", connection)

In [161]:
df_odds

,match_id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,outcome_name,outcome_price
0,dcf1380413740a8f8ee4c016c8550ea8,soccer_epl,EPL,2024-08-31 14:00:00,Everton,Bournemouth,suprabets,Suprabets,2024-08-23 18:37:32,h2h,2024-08-23 18:37:32,Draw,3.60
1,fc3ce4f31b7fe23e3d8eccd0c12ebfbf,soccer_epl,EPL,2024-09-01 12:30:00,Chelsea,Crystal Palace,betsson,Betsson,2024-08-23 18:39:22,h2h,2024-08-23 18:39:22,Chelsea,1.72
2,5c835569c5bedfa3ef295b5ef1118dbf,soccer_epl,EPL,2024-08-31 14:00:00,Brentford,Southampton,unibet_eu,Unibet,2024-08-23 18:39:19,h2h,2024-08-23 18:39:19,Draw,3.80
3,301c26724a820ad587e4973ea78d3971,soccer_france_ligue_one,Ligue 1 - France,2024-08-30 18:45:00,Lyon,Strasbourg,pinnacle,Pinnacle,2024-08-23 18:38:04,h2h,2024-08-23 18:38:04,Lyon,1.49
4,301c26724a820ad587e4973ea78d3971,soccer_france_ligue_one,Ligue 1 - France,2024-08-30 18:45:00,Lyon,Strasbourg,pinnacle,Pinnacle,2024-08-23 18:38:04,h2h,2024-08-23 18:38:04,Strasbourg,5.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,eb749753d96a4af7d4a56f949573a279,soccer_epl,EPL,2024-09-01 15:00:00,Manchester United,Liverpool,matchbook,Matchbook,2024-08-26 13:55:10,h2h_lay,2024-08-26 13:55:10,Manchester United,4.10
5129,eb749753d96a4af7d4a56f949573a279,soccer_epl,EPL,2024-09-01 15:00:00,Manchester United,Liverpool,matchbook,Matchbook,2024-08-26 13:55:10,h2h_lay,2024-08-26 13:55:10,Draw,4.40
5130,eb749753d96a4af7d4a56f949573a279,soccer_epl,EPL,2024-09-01 15:00:00,Manchester United,Liverpool,betonlineag,BetOnline.ag,2024-08-26 13:54:04,h2h,2024-08-26 13:54:04,Liverpool,1.83
5131,eb749753d96a4af7d4a56f949573a279,soccer_epl,EPL,2024-09-01 15:00:00,Manchester United,Liverpool,betonlineag,BetOnline.ag,2024-08-26 13:54:04,h2h,2024-08-26 13:54:04,Manchester United,3.90


In [162]:
df_odds__last_odds = df_odds.sort_values(by=['commence_time', 'match_id', 'bookmaker_key', 'bookmaker_last_update'], ascending=False)
df_odds__last_odds = df_odds__last_odds.drop_duplicates(subset=['match_id', 'bookmaker_key', 'outcome_name'], keep='first')
df_odds__last_odds

,match_id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,outcome_name,outcome_price
3397,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,williamhill,William Hill,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Real Betis,9.00
3398,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,williamhill,William Hill,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Real Madrid,1.33
3399,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,williamhill,William Hill,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Draw,5.50
3391,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,unibet_eu,Unibet,2024-08-26 13:55:49,h2h,2024-08-26 13:55:49,Real Betis,8.75
3392,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,unibet_eu,Unibet,2024-08-26 13:55:49,h2h,2024-08-26 13:55:49,Real Madrid,1.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betfair_ex_eu,Betfair,2024-08-26 13:55:50,h2h,2024-08-26 13:55:50,Como,3.45
3420,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betfair_ex_eu,Betfair,2024-08-26 13:55:50,h2h,2024-08-26 13:55:50,Draw,3.40
3412,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betclic,Betclic,2024-08-26 13:55:50,h2h,2024-08-26 13:55:50,Cagliari,2.24
3413,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betclic,Betclic,2024-08-26 13:55:50,h2h,2024-08-26 13:55:50,Como,3.22


In [163]:
df_odds__last_odds__home = df_odds__last_odds[df_odds__last_odds['outcome_name'] == df_odds__last_odds['home_team']]
df_odds__last_odds__away = df_odds__last_odds[df_odds__last_odds['outcome_name'] == df_odds__last_odds['away_team']]
df_odds__last_odds__draw = df_odds__last_odds[df_odds__last_odds['outcome_name'] == 'Draw']
df_odds__last_odds__home = df_odds__last_odds__home.rename(columns={'outcome_price': 'odds_home'})
df_odds__last_odds__away = df_odds__last_odds__away.rename(columns={'outcome_price': 'odds_away'})
df_odds__last_odds__draw = df_odds__last_odds__draw.rename(columns={'outcome_price': 'odds_draw'})

df_odds__last_odds__draw = df_odds__last_odds__draw[['match_id', 'bookmaker_key', 'odds_draw']]
df_odds__last_odds__away = df_odds__last_odds__away[['match_id', 'bookmaker_key', 'odds_away']]

df_odds__last_odds__home_draw = pd.merge(df_odds__last_odds__home, df_odds__last_odds__draw, on=['match_id', 'bookmaker_key'], how='inner')
df_odds__last_odds__home_draw_away = pd.merge(df_odds__last_odds__home_draw, df_odds__last_odds__away, on=['match_id', 'bookmaker_key'], how='inner')
df_odds__last_odds__home_draw_away

,match_id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,outcome_name,odds_home,odds_draw,odds_away
0,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,williamhill,William Hill,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Real Madrid,1.33,5.50,9.00
1,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,unibet_eu,Unibet,2024-08-26 13:55:49,h2h,2024-08-26 13:55:49,Real Madrid,1.30,5.60,8.75
2,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,tipico_de,Tipico,2024-08-26 13:55:22,h2h,2024-08-26 13:55:22,Real Madrid,1.30,5.80,8.50
3,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,suprabets,Suprabets,2024-08-26 13:53:03,h2h,2024-08-26 13:53:03,Real Madrid,1.36,6.00,8.50
4,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,sport888,888sport,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Real Madrid,1.30,5.50,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,coolbet,Coolbet,2024-08-26 13:55:21,h2h,2024-08-26 13:55:21,Cagliari,2.31,3.35,3.35
951,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betsson,Betsson,2024-08-26 13:55:50,h2h,2024-08-26 13:55:50,Cagliari,2.28,3.35,3.15
952,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betonlineag,BetOnline.ag,2024-08-26 13:55:21,h2h,2024-08-26 13:55:21,Cagliari,2.31,3.30,3.35
953,f3cb6b728afd9c9c28fe6aeb9b5d2038,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,betfair_ex_eu,Betfair,2024-08-26 13:55:50,h2h,2024-08-26 13:55:50,Cagliari,2.36,3.40,3.45


In [128]:
df_odds__last_odds__home_draw_away

,match_id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,outcome_name,odds_home,odds_draw,odds_away
0,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,williamhill,William Hill,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Real Madrid,1.33,5.50,9.00
1,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,unibet_eu,Unibet,2024-08-26 13:55:49,h2h,2024-08-26 13:55:49,Real Madrid,1.30,5.60,8.75
2,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,tipico_de,Tipico,2024-08-26 13:55:22,h2h,2024-08-26 13:55:22,Real Madrid,1.30,5.80,8.50
3,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,suprabets,Suprabets,2024-08-26 13:53:03,h2h,2024-08-26 13:53:03,Real Madrid,1.36,6.00,8.50
4,d32ac1b4944230ac2c96ffe41d6197ba,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,sport888,888sport,2024-08-26 13:55:48,h2h,2024-08-26 13:55:48,Real Madrid,1.30,5.50,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,c3f950a4b9e19115303fbbe760ebc8da,soccer_uefa_europa_conference_league,UEFA Europa Conference League,2024-07-25 18:45:00,Víkingur Reykjavík,KF Egnatia,nordicbet,Nordic Bet,2024-07-25 10:50:10,h2h,2024-07-25 10:50:10,Víkingur Reykjavík,1.72,3.75,4.50
1947,c3f950a4b9e19115303fbbe760ebc8da,soccer_uefa_europa_conference_league,UEFA Europa Conference League,2024-07-25 18:45:00,Víkingur Reykjavík,KF Egnatia,mybookieag,MyBookie.ag,2024-07-25 10:50:11,h2h,2024-07-25 10:50:11,Víkingur Reykjavík,1.74,3.85,4.40
1948,c3f950a4b9e19115303fbbe760ebc8da,soccer_uefa_europa_conference_league,UEFA Europa Conference League,2024-07-25 18:45:00,Víkingur Reykjavík,KF Egnatia,marathonbet,Marathon Bet,2024-07-25 10:50:10,h2h,2024-07-25 10:50:10,Víkingur Reykjavík,1.73,3.80,4.40
1949,c3f950a4b9e19115303fbbe760ebc8da,soccer_uefa_europa_conference_league,UEFA Europa Conference League,2024-07-25 18:45:00,Víkingur Reykjavík,KF Egnatia,betsson,Betsson,2024-07-25 10:47:57,h2h,2024-07-25 10:47:57,Víkingur Reykjavík,1.72,3.75,4.50


In [164]:
max_odds_df = df_odds__last_odds__home_draw_away.groupby('match_id').agg({
    'odds_home': 'max',
    'odds_draw': 'max',
    'odds_away': 'max'
}).reset_index()

# Adding columns to store the bookmaker who provided the highest odds for each outcome
max_odds_df['bookmaker_home'] = max_odds_df.apply(lambda x: df_odds__last_odds__home_draw_away[(df_odds__last_odds__home_draw_away['match_id'] == x['match_id']) & (df_odds__last_odds__home_draw_away['odds_home'] == x['odds_home'])]['bookmaker_title'].values[0], axis=1)
max_odds_df['bookmaker_draw'] = max_odds_df.apply(lambda x: df_odds__last_odds__home_draw_away[(df_odds__last_odds__home_draw_away['match_id'] == x['match_id']) & (df_odds__last_odds__home_draw_away['odds_draw'] == x['odds_draw'])]['bookmaker_title'].values[0], axis=1)
max_odds_df['bookmaker_away'] = max_odds_df.apply(lambda x: df_odds__last_odds__home_draw_away[(df_odds__last_odds__home_draw_away['match_id'] == x['match_id']) & (df_odds__last_odds__home_draw_away['odds_away'] == x['odds_away'])]['bookmaker_title'].values[0], axis=1)

df_odds__last_odds__home_draw_away__no_bookie_col = df_odds__last_odds__home_draw_away.drop(columns=['bookmaker_title', 'bookmaker_key', 'bookmaker_last_update', 'market_key', 'market_last_update', 'outcome_name', 'odds_home', 'odds_draw', 'odds_away'])
max_odds_df_all = max_odds_df.merge(df_odds__last_odds__home_draw_away__no_bookie_col, on='match_id', how='outer')
max_odds_df_all = max_odds_df_all.drop_duplicates(subset=['match_id'], keep='first')
max_odds_df_all.sort_values(by='commence_time', ascending=False)


,match_id,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,home_team,away_team
703,d32ac1b4944230ac2c96ffe41d6197ba,1.36,6.00,10.00,Suprabets,Suprabets,888sport,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis
716,d5d6942126b2ff11e4a987fa157dbfb9,1.99,3.66,4.33,1xBet,1xBet,888sport,soccer_italy_serie_a,Serie A - Italy,2024-09-01 18:45:00,Juventus,AS Roma
246,56a0098bce42a4ab8340ca6a87913004,1.94,3.73,4.50,1xBet,1xBet,Unibet,soccer_italy_serie_a,Serie A - Italy,2024-09-01 18:45:00,Udinese,Como
487,96d220ad3c9df498d4bd1d2cfb5ec6e0,3.86,3.85,2.05,Suprabets,1xBet,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 18:45:00,Lille,Paris Saint Germain
549,a2781a710bdfac8ed30574de2f6a0ed8,3.50,3.10,2.56,Tipico,1xBet,Betfair,soccer_spain_la_liga,La Liga - Spain,2024-09-01 17:15:00,Getafe,Real Sociedad
...,...,...,...,...,...,...,...,...,...,...,...,...
938,ff98a44f5d6c74872d4993b82420a6f9,5.20,4.20,1.75,Matchbook,Matchbook,Suprabets,soccer_spain_la_liga,La Liga - Spain,2024-08-27 19:30:00,Rayo Vallecano,Barcelona
655,c4fca039c3f4797190006b4d0344ad06,2.46,3.29,3.55,Matchbook,1xBet,Unibet,soccer_spain_la_liga,La Liga - Spain,2024-08-27 17:00:00,Mallorca,Sevilla
470,9453b9271c4513f6f2e329fe638a98b1,1.85,4.10,4.60,William Hill,Unibet,Matchbook,soccer_spain_la_liga,La Liga - Spain,2024-08-26 19:30:00,Villarreal,Celta Vigo
386,76030a8f2e63f847b2ca42ae96374de9,5.20,3.73,1.85,Matchbook,1xBet,1xBet,soccer_italy_serie_a,Serie A - Italy,2024-08-26 18:45:00,Hellas Verona,Juventus


In [130]:
max_odds_df_all['1/sum(odds)'] = 1/max_odds_df_all['odds_home'] + 1/max_odds_df_all['odds_draw'] + 1/max_odds_df_all['odds_away']
max_odds_df_all.sort_values(by='1/sum(odds)', ascending=True)

,match_id,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,home_team,away_team,1/sum(odds)
600,5a62b74a152daecad27d6b5856270d2c,2.17,29.00,310.00,1xBet,Suprabets,Matchbook,soccer_spain_la_liga,La Liga - Spain,2024-08-23 17:00:25,Celta Vigo,Valencia,0.498538
976,95febdd561d0823b4db430c76298f9e2,6.00,4.62,1.74,Matchbook,1xBet,Betfair,soccer_germany_bundesliga,Bundesliga - Germany,2024-08-23 18:30:00,Borussia Monchengladbach,Bayer Leverkusen,0.957830
1457,d229c4c4cbec8cdbe3c00ffdede01223,2.24,3.60,4.16,Betfair,1xBet,Pinnacle,soccer_turkey_super_league,Turkey Super League,2024-08-23 18:00:00,Alanyaspor,Goztepe,0.964591
1181,a425f973fa2ebf57794fdd9df46e1aec,2.44,3.54,3.40,Matchbook,1xBet,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-08-25 15:00:00,Nantes,Auxerre,0.986440
699,6c06a19c9688ebf4401305868213aa0c,2.58,3.75,3.00,Suprabets,Unibet,Matchbook,soccer_germany_bundesliga,Bundesliga - Germany,2024-09-01 13:30:00,1. FC Heidenheim,Augsburg,0.987597
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,3767eafd91c3621342db8bb5c8620816,2.12,3.40,3.95,Betfair,Betsson,Betfair,soccer_spain_la_liga,La Liga - Spain,2024-09-01 15:00:00,Alavés,Las Palmas,1.018980
1800,f4da5ed646a0d211a1b009f959ebb8a0,1.71,3.65,6.20,Pinnacle,Betfair,Betfair,soccer_brazil_serie_b,Brazil Série B,2024-07-26 00:30:00,Ceará,Botafogo,1.020058
1029,983a35cd99075c2ac996dd7c3e6f0a59,1.72,3.75,5.50,Nordic Bet,Nordic Bet,888sport,soccer_spain_la_liga,La Liga - Spain,2024-08-28 16:00:00,Real Betis,Getafe,1.029880
247,2c3b6478d0b0f9567e0c3cabdad4faf1,1.73,3.67,5.38,Pinnacle,Pinnacle,Pinnacle,soccer_spain_la_liga,La Liga - Spain,2024-08-28 00:00:00,Real Betis,Getafe,1.036388


In [171]:
sports = ['soccer_france_ligue_one', 'soccer_spain_la_liga', 'soccer_italy_serie_a', 'soccer_germany_bundesliga', 'soccer_epl']
max_odds_df_all_big5 = max_odds_df_all[max_odds_df_all['sport_key'].isin(sports)]
max_odds_df_all_big5

,match_id,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,home_team,away_team
0,055f48ea54ea77ffa2b16ef7310b04b6,1.36,6.50,9.80,Unibet,Suprabets,Matchbook,soccer_epl,EPL,2024-08-31 11:30:00,Arsenal,Brighton and Hove Albion
18,06cbeb0e6a762b05a5f1af4e955d4917,3.02,3.62,2.57,Suprabets,1xBet,1xBet,soccer_italy_serie_a,Serie A - Italy,2024-08-31 18:45:00,Lazio,AC Milan
34,1cd07b9727db86afe232f2aa158adbc3,4.00,4.30,1.93,Tipico,1xBet,BetOnline.ag,soccer_germany_bundesliga,Bundesliga - Germany,2024-08-31 13:30:00,Werder Bremen,Borussia Dortmund
50,1d2dc8be4a4e31dd4dfa8cb7f8947258,2.32,3.43,3.50,Suprabets,1xBet,Suprabets,soccer_spain_la_liga,La Liga - Spain,2024-09-01 15:00:00,CA Osasuna,Celta Vigo
63,296960a0ba48de8f014c949753ccf40e,1.76,4.16,5.20,Matchbook,1xBet,Unibet,soccer_spain_la_liga,La Liga - Spain,2024-08-29 17:00:00,Girona,CA Osasuna
...,...,...,...,...,...,...,...,...,...,...,...,...
868,f3cb6b728afd9c9c28fe6aeb9b5d2038,2.38,3.54,3.45,Matchbook,1xBet,Betfair,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como
886,f652867f8b3a00096122e76b4e554e4c,1.64,4.20,7.00,Suprabets,Unibet,Matchbook,soccer_spain_la_liga,La Liga - Spain,2024-08-28 17:00:00,Athletic Bilbao,Valencia
903,fb3014809c599153afdd9f5d738b58e6,2.38,3.17,3.95,Matchbook,1xBet,Matchbook,soccer_spain_la_liga,La Liga - Spain,2024-08-28 17:00:00,Valladolid,Leganés
920,fc3ce4f31b7fe23e3d8eccd0c12ebfbf,1.68,4.50,5.20,Suprabets,Suprabets,Suprabets,soccer_epl,EPL,2024-09-01 12:30:00,Chelsea,Crystal Palace


In [166]:
set_team_name_odds_df = set(list(max_odds_df_all_big5['home_team'].unique()) + list(max_odds_df_all_big5['away_team'].unique()))
set_team_name_models_results_df = set(list(df_models_results['home_team'].unique()) + list(df_models_results['away_team'].unique()))

In [167]:
is_not_in_odds = set_team_name_models_results_df - set_team_name_odds_df
is_not_in_odds

{'Atalanta',
 'Athletic Club',
 'Bochum',
 'Brighton',
 'Dortmund',
 'Eint Frankfurt',
 'Freiburg',
 'Gladbach',
 'Heidenheim',
 'Hoffenheim',
 'Inter',
 'Lens',
 'Leverkusen',
 'Mainz 05',
 'Manchester Utd',
 'Milan',
 'Monaco',
 'Newcastle Utd',
 "Nott'ham Forest",
 'Osasuna',
 'Paris S-G',
 'Roma',
 'Saint-Étienne',
 'St. Pauli',
 'Stuttgart',
 'Tottenham',
 'West Ham',
 'Wolfsburg',
 'Wolves'}

In [168]:
is_not_in_models_results = set_team_name_odds_df - set_team_name_models_results_df
is_not_in_models_results

{'1. FC Heidenheim',
 'AC Milan',
 'AS Monaco',
 'AS Roma',
 'Atalanta BC',
 'Athletic Bilbao',
 'Bayer Leverkusen',
 'Borussia Dortmund',
 'Borussia Monchengladbach',
 'Brighton and Hove Albion',
 'CA Osasuna',
 'Como',
 'Eintracht Frankfurt',
 'FC St. Pauli',
 'FSV Mainz 05',
 'Holstein Kiel',
 'Inter Milan',
 'Ipswich Town',
 'Manchester United',
 'Newcastle United',
 'Nottingham Forest',
 'Paris Saint Germain',
 'RC Lens',
 'Real Betis',
 'SC Freiburg',
 'Saint Etienne',
 'Stade de Reims',
 'TSG Hoffenheim',
 'Tottenham Hotspur',
 'VfB Stuttgart',
 'VfL Bochum',
 'VfL Wolfsburg',
 'West Ham United',
 'Wolverhampton Wanderers'}

In [169]:
mapping_dict = {
    '1. FC Heidenheim': 'Heidenheim',
    'AC Milan': 'Milan',
    'AS Monaco': 'Monaco',
    'AS Roma': 'Roma',
    'Atalanta BC': 'Atalanta',
    'Athletic Bilbao': 'Athletic Club',
    'Bayer Leverkusen': 'Leverkusen',
    'Borussia Dortmund': 'Dortmund',
    'Borussia Monchengladbach': 'Gladbach',
    'Brighton and Hove Albion': 'Brighton',
    'CA Osasuna': 'Osasuna',
    'Eintracht Frankfurt': 'Eint Frankfurt',
    'FC St. Pauli': 'St. Pauli',
    'FSV Mainz 05': 'Mainz 05',
    'Inter Milan': 'Inter',
    'Manchester United': 'Manchester Utd',
    'Newcastle United': 'Newcastle Utd',
    'Nottingham Forest': "Nott'ham Forest",
    'Paris Saint Germain': 'Paris S-G',
    'RC Lens': 'Lens',
    'SC Freiburg': 'Freiburg',
    'Saint Etienne': 'Saint-Étienne',
    'TSG Hoffenheim': 'Hoffenheim',
    'Tottenham Hotspur': 'Tottenham',
    'VfB Stuttgart': 'Stuttgart',
    'VfL Bochum': 'Bochum',
    'VfL Wolfsburg': 'Wolfsburg',
    'West Ham United': 'West Ham',
    'Wolverhampton Wanderers': 'Wolves'
}


In [172]:
max_odds_df_all_mapped = max_odds_df_all_big5.replace({'home_team': mapping_dict, 'away_team': mapping_dict})
max_odds_df_all_mapped

,match_id,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,home_team,away_team
0,055f48ea54ea77ffa2b16ef7310b04b6,1.36,6.50,9.80,Unibet,Suprabets,Matchbook,soccer_epl,EPL,2024-08-31 11:30:00,Arsenal,Brighton
18,06cbeb0e6a762b05a5f1af4e955d4917,3.02,3.62,2.57,Suprabets,1xBet,1xBet,soccer_italy_serie_a,Serie A - Italy,2024-08-31 18:45:00,Lazio,Milan
34,1cd07b9727db86afe232f2aa158adbc3,4.00,4.30,1.93,Tipico,1xBet,BetOnline.ag,soccer_germany_bundesliga,Bundesliga - Germany,2024-08-31 13:30:00,Werder Bremen,Dortmund
50,1d2dc8be4a4e31dd4dfa8cb7f8947258,2.32,3.43,3.50,Suprabets,1xBet,Suprabets,soccer_spain_la_liga,La Liga - Spain,2024-09-01 15:00:00,Osasuna,Celta Vigo
63,296960a0ba48de8f014c949753ccf40e,1.76,4.16,5.20,Matchbook,1xBet,Unibet,soccer_spain_la_liga,La Liga - Spain,2024-08-29 17:00:00,Girona,Osasuna
...,...,...,...,...,...,...,...,...,...,...,...,...
868,f3cb6b728afd9c9c28fe6aeb9b5d2038,2.38,3.54,3.45,Matchbook,1xBet,Betfair,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como
886,f652867f8b3a00096122e76b4e554e4c,1.64,4.20,7.00,Suprabets,Unibet,Matchbook,soccer_spain_la_liga,La Liga - Spain,2024-08-28 17:00:00,Athletic Club,Valencia
903,fb3014809c599153afdd9f5d738b58e6,2.38,3.17,3.95,Matchbook,1xBet,Matchbook,soccer_spain_la_liga,La Liga - Spain,2024-08-28 17:00:00,Valladolid,Leganés
920,fc3ce4f31b7fe23e3d8eccd0c12ebfbf,1.68,4.50,5.20,Suprabets,Suprabets,Suprabets,soccer_epl,EPL,2024-09-01 12:30:00,Chelsea,Crystal Palace


In [173]:
set_team_name_odds_mapped_df = set(list(max_odds_df_all_mapped['home_team'].unique()) + list(max_odds_df_all_mapped['away_team'].unique()))
set_team_name_models_results_df = set(list(df_models_results['home_team'].unique()) + list(df_models_results['away_team'].unique()))
set_team_name_odds_mapped_df - set_team_name_models_results_df

{'Como', 'Holstein Kiel', 'Ipswich Town', 'Real Betis', 'Stade de Reims'}

In [174]:
set_team_name_models_results_df - set_team_name_odds_mapped_df

set()

In [176]:
df_models_results_last_infered = df_models_results.sort_values(by=['datetime_inference'], ascending=False).drop_duplicates(subset=['game'], keep='first')
df_models_results_last_infered                                          

,datetime_inference,model,game_id,game,date_match,time_match,home_team,away_team,prob_home_win,prob_draw,prob_away_win,prob_home_team_score,prob_away_team_score
7456,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2025-05-25 Rayo Vallecano-Mallorca,2025-05-25,None,Rayo Vallecano,Mallorca,0.344962,0.311249,0.343789,None,None
6467,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-12-03 Aston Villa-Brentford,2024-12-03,19:45:00,Aston Villa,Brentford,0.436960,0.261388,0.301652,None,None
6458,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-12-01 Rennes-Saint-Étienne,2024-12-01,None,Rennes,Saint-Étienne,0.454508,0.284271,0.261221,None,None
6459,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-12-01 Toulouse-Auxerre,2024-12-01,None,Toulouse,Auxerre,0.349314,0.307001,0.343685,None,None
6460,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-12-01 Alavés-Leganés,2024-12-01,None,Alavés,Leganés,0.319122,0.317985,0.362893,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6959,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2025-03-01 Stuttgart-Bayern Munich,2025-03-01,None,Stuttgart,Bayern Munich,0.124572,0.184966,0.690462,None,None
4477,2024-08-26 14:09:55.651611,RSF_PR_LR,None,2024-08-27 Mallorca-Sevilla,2024-08-27,19:00:00,Mallorca,Sevilla,0.262620,0.306880,0.430500,None,None
4478,2024-08-26 14:09:55.651611,RSF_PR_LR,None,2024-08-27 Rayo Vallecano-Barcelona,2024-08-27,21:30:00,Rayo Vallecano,Barcelona,0.113179,0.198003,0.688819,None,None
1494,2024-08-25 06:32:38.485779,RSF_PR_LR,None,2024-08-26 Hellas Verona-Juventus,2024-08-26,20:45:00,Hellas Verona,Juventus,0.113455,0.218883,0.667662,None,None


In [177]:
max_odds_df_all_mapped['date_match'] = max_odds_df_all_mapped['commence_time'].dt.date
max_odds_df_all_mapped.sort_values(by='date_match', ascending=False).head(10)

,match_id,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,home_team,away_team,date_match
230,544aeaeb39321c6393d765e392df93fe,2.76,3.52,2.75,Suprabets,Suprabets,William Hill,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 15:00:00,Le Havre,Auxerre,2024-09-01
640,bf7ceb38c2ae5c21b06b123f47ae2277,1.23,8.50,15.00,Suprabets,Unibet,Unibet,soccer_germany_bundesliga,Bundesliga - Germany,2024-09-01 15:30:00,Bayern Munich,Freiburg,2024-09-01
509,99d6741f20aaffccb6cb6cfc436a4ad3,2.20,3.40,3.93,Suprabets,1xBet,Suprabets,soccer_italy_serie_a,Serie A - Italy,2024-09-01 16:30:00,Genoa,Hellas Verona,2024-09-01
536,9f19ea203d52e54831c8f7a6cd904fdb,2.44,3.95,2.90,1xBet,Suprabets,1xBet,soccer_spain_la_liga,La Liga - Spain,2024-09-01 17:00:00,Sevilla,Girona,2024-09-01
920,fc3ce4f31b7fe23e3d8eccd0c12ebfbf,1.68,4.50,5.20,Suprabets,Suprabets,Suprabets,soccer_epl,EPL,2024-09-01 12:30:00,Chelsea,Crystal Palace,2024-09-01
549,a2781a710bdfac8ed30574de2f6a0ed8,3.50,3.10,2.56,Tipico,1xBet,Betfair,soccer_spain_la_liga,La Liga - Spain,2024-09-01 17:15:00,Getafe,Real Sociedad,2024-09-01
562,a371f7970611f76231c816ba2be8bc4b,3.95,3.52,2.14,Betfair,1xBet,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 15:00:00,Angers,Nice,2024-09-01
340,6c06a19c9688ebf4401305868213aa0c,2.58,3.75,3.00,Suprabets,Unibet,Matchbook,soccer_germany_bundesliga,Bundesliga - Germany,2024-09-01 13:30:00,Heidenheim,Augsburg,2024-09-01
607,af20930f748ec57eec5478c729ac66dd,1.89,4.20,4.20,Betfair,Suprabets,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 13:00:00,Monaco,Lens,2024-09-01
278,5856ec2bd21512f6f5aee4e9160b2fb3,1.65,4.30,6.50,Betfair,Unibet,Unibet,soccer_italy_serie_a,Serie A - Italy,2024-09-01 16:30:00,Fiorentina,Monza,2024-09-01


In [182]:
df_models_results

,datetime_inference,model,game_id,game,date_match,time_match,home_team,away_team,prob_home_win,prob_draw,prob_away_win,prob_home_team_score,prob_away_team_score
0,2024-08-24 06:32:29.161615,RSF_PR_LR,None,2024-08-26 Villarreal-Celta Vigo,2024-08-26,21:30:00,Villarreal,Celta Vigo,0.458853,0.281917,0.259230,None,None
1,2024-08-24 06:32:29.161615,RSF_PR_LR,None,2024-08-26 Hellas Verona-Juventus,2024-08-26,20:45:00,Hellas Verona,Juventus,0.113455,0.218883,0.667662,None,None
2,2024-08-24 06:32:29.161615,RSF_PR_LR,None,2024-08-27 Rayo Vallecano-Barcelona,2024-08-27,21:30:00,Rayo Vallecano,Barcelona,0.111840,0.200223,0.687937,None,None
3,2024-08-24 06:32:29.161615,RSF_PR_LR,None,2024-08-27 Mallorca-Sevilla,2024-08-27,19:00:00,Mallorca,Sevilla,0.259567,0.307922,0.432511,None,None
4,2024-08-24 06:32:29.161615,RSF_PR_LR,None,2024-08-28 Real Sociedad-Alavés,2024-08-28,21:30:00,Real Sociedad,Alavés,0.557526,0.245162,0.197313,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7452,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2025-05-25 Newcastle Utd-Everton,2025-05-25,16:00:00,Newcastle Utd,Everton,0.447012,0.271027,0.281961,None,None
7453,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2025-05-25 Manchester Utd-Aston Villa,2025-05-25,16:00:00,Manchester Utd,Aston Villa,0.451184,0.290135,0.258681,None,None
7454,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2025-05-25 Liverpool-Crystal Palace,2025-05-25,16:00:00,Liverpool,Crystal Palace,0.623853,0.231268,0.144879,None,None
7455,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2025-05-25 Empoli-Hellas Verona,2025-05-25,None,Empoli,Hellas Verona,0.308939,0.323256,0.367805,None,None


In [187]:
df_models_results_last_infered.describe()

,datetime_inference,prob_home_win,prob_draw,prob_away_win
count,1493,1493.000000,1493.000000,1493.000000
mean,2024-08-27 06:27:42.146550272,0.363111,0.260843,0.376047
min,2024-08-25 06:32:38.485779,0.061200,0.135379,0.054193
25%,2024-08-27 06:32:52.650452992,0.242519,0.231343,0.256382
50%,2024-08-27 06:32:52.650452992,0.347823,0.269751,0.356535
75%,2024-08-27 06:32:52.650452992,0.467291,0.292506,0.486280
max,2024-08-27 06:32:52.650453,0.789376,0.350281,0.803421
std,NaN,0.161020,0.040806,0.164996


In [188]:
max_odds_df_all_mapped.describe()

,odds_home,odds_draw,odds_away,commence_time
count,61.00000,61.000000,61.000000,61
mean,2.66459,4.130984,4.291967,2024-08-31 04:16:43.278688512
min,1.23000,3.100000,1.440000,2024-08-26 16:30:00
25%,1.76000,3.620000,2.570000,2024-08-30 18:45:00
50%,2.29000,3.850000,3.700000,2024-08-31 16:30:00
75%,3.02000,4.300000,5.200000,2024-09-01 13:30:00
max,8.40000,8.500000,15.000000,2024-09-01 19:30:00
std,1.37155,0.937583,2.660884,NaN


In [186]:
df_models_results_joined = df_models_results_last_infered.merge(max_odds_df_all_mapped, left_on=['home_team', 'away_team', 'date_match'], right_on=['home_team', 'away_team', 'date_match'], how='inner')
df_models_results_joined.sort_values(by='date_match', ascending=False)

,datetime_inference,model,game_id,game,date_match,time_match,home_team,away_team,prob_home_win,prob_draw,...,match_id,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time
44,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Monaco-Lens,2024-09-01,15:00:00,Monaco,Lens,0.412260,0.270235,...,af20930f748ec57eec5478c729ac66dd,1.89,4.20,4.20,Betfair,Suprabets,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 13:00:00
12,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Bayern Munich-Freiburg,2024-09-01,17:30:00,Bayern Munich,Freiburg,0.685648,0.191255,...,bf7ceb38c2ae5c21b06b123f47ae2277,1.23,8.50,15.00,Suprabets,Unibet,Unibet,soccer_germany_bundesliga,Bundesliga - Germany,2024-09-01 15:30:00
48,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Juventus-Roma,2024-09-01,20:45:00,Juventus,Roma,0.423766,0.244631,...,d5d6942126b2ff11e4a987fa157dbfb9,1.99,3.66,4.33,1xBet,1xBet,888sport,soccer_italy_serie_a,Serie A - Italy,2024-09-01 18:45:00
46,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Fiorentina-Monza,2024-09-01,18:30:00,Fiorentina,Monza,0.457891,0.273471,...,5856ec2bd21512f6f5aee4e9160b2fb3,1.65,4.30,6.50,Betfair,Unibet,Unibet,soccer_italy_serie_a,Serie A - Italy,2024-09-01 16:30:00
45,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Getafe-Real Sociedad,2024-09-01,19:15:00,Getafe,Real Sociedad,0.189275,0.258223,...,a2781a710bdfac8ed30574de2f6a0ed8,3.50,3.10,2.56,Tipico,1xBet,Betfair,soccer_spain_la_liga,La Liga - Spain,2024-09-01 17:15:00
19,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Newcastle Utd-Tottenham,2024-09-01,13:30:00,Newcastle Utd,Tottenham,0.257747,0.220531,...,36c348ee3ab335d09cabd3a112caaa4a,2.60,4.33,2.64,Suprabets,Suprabets,Matchbook,soccer_epl,EPL,2024-09-01 12:30:00
18,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Manchester Utd-Liverpool,2024-09-01,16:00:00,Manchester Utd,Liverpool,0.304763,0.262971,...,eb749753d96a4af7d4a56f949573a279,4.00,4.40,1.90,Tipico,Suprabets,1xBet,soccer_epl,EPL,2024-09-01 15:00:00
17,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Chelsea-Crystal Palace,2024-09-01,13:30:00,Chelsea,Crystal Palace,0.507629,0.262991,...,fc3ce4f31b7fe23e3d8eccd0c12ebfbf,1.68,4.50,5.20,Suprabets,Suprabets,Suprabets,soccer_epl,EPL,2024-09-01 12:30:00
16,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Heidenheim-Augsburg,2024-09-01,15:30:00,Heidenheim,Augsburg,0.340747,0.304397,...,6c06a19c9688ebf4401305868213aa0c,2.58,3.75,3.00,Suprabets,Unibet,Matchbook,soccer_germany_bundesliga,Bundesliga - Germany,2024-09-01 13:30:00
15,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-09-01 Lille-Paris S-G,2024-09-01,20:45:00,Lille,Paris S-G,0.172167,0.184041,...,96d220ad3c9df498d4bd1d2cfb5ec6e0,3.86,3.85,2.05,Suprabets,1xBet,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 18:45:00


In [196]:
df_left_join = max_odds_df_all_mapped.merge(df_models_results_last_infered, left_on=['home_team', 'away_team', 'date_match'], right_on=['home_team', 'away_team', 'date_match'], how='left', indicator=True)
df_not_in_merged = df_left_join[df_left_join['_merge'] == 'left_only'].sort_values(by='date_match', ascending=False)
df_not_in_merged[['odds_home', 'odds_draw', 'odds_away', 'bookmaker_home',
       'bookmaker_draw', 'bookmaker_away', 'sport_key', 'sport_title',
       'commence_time', 'home_team', 'away_team', 'date_match',
       'datetime_inference', 'model',
       'prob_home_win', 'prob_draw', 'prob_away_win', 'prob_home_team_score',
       'prob_away_team_score']]

,odds_home,odds_draw,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,home_team,away_team,date_match,datetime_inference,model,prob_home_win,prob_draw,prob_away_win,prob_home_team_score,prob_away_team_score
16,1.94,3.73,4.50,1xBet,1xBet,Unibet,soccer_italy_serie_a,Serie A - Italy,2024-09-01 18:45:00,Udinese,Como,2024-09-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN
29,3.05,3.71,2.50,Suprabets,1xBet,Unibet,soccer_france_ligue_one,Ligue 1 - France,2024-09-01 15:00:00,Stade de Reims,Rennes,2024-09-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN
46,1.36,6.00,10.00,Suprabets,Suprabets,888sport,soccer_spain_la_liga,La Liga - Spain,2024-09-01 19:30:00,Real Madrid,Real Betis,2024-09-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN
13,3.10,3.75,2.43,Tipico,1xBet,Suprabets,soccer_epl,EPL,2024-08-31 14:00:00,Ipswich Town,Fulham,2024-08-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN
54,3.35,3.80,2.23,Suprabets,Unibet,Unibet,soccer_germany_bundesliga,Bundesliga - Germany,2024-08-31 13:30:00,Holstein Kiel,Wolfsburg,2024-08-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5,1.73,3.67,5.38,Pinnacle,Pinnacle,Pinnacle,soccer_spain_la_liga,La Liga - Spain,2024-08-28 00:00:00,Real Betis,Getafe,2024-08-28,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32,1.72,3.75,5.50,Nordic Bet,Nordic Bet,888sport,soccer_spain_la_liga,La Liga - Spain,2024-08-28 16:00:00,Real Betis,Getafe,2024-08-28,NaT,NaN,NaN,NaN,NaN,NaN,NaN
56,2.38,3.54,3.45,Matchbook,1xBet,Betfair,soccer_italy_serie_a,Serie A - Italy,2024-08-26 16:30:00,Cagliari,Como,2024-08-26,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
o = df_models_results_joined[['odds_home', 'odds_draw', 'odds_away']].to_numpy()
r = df_models_results_joined[['prob_home_win', 'prob_draw', 'prob_away_win']].to_numpy()


In [201]:
import numpy as np

from optim.functions.player_gain_expected_value import player_gain_expected_value
from optim.functions.player_gain_variance import player_gain_variance
from optim.functions.player_utility_kelly_criteria import player_utility_kelly_criteria
from optim.functions.player_utility_linear import player_utility_linear
from optim.functions.player_utility_sharp_ratio import player_utility_sharp_ratio

from optim.resolve.resolve_fik import resolve_fik

print("Cotes:\n", o)
print("Probabilités:\n", r)

Cotes:
 [[ 1.85  4.3   4.6 ]
 [ 2.74  3.21  3.1 ]
 [ 1.31  6.5  11.  ]
 [ 3.    3.47  2.57]
 [ 4.7   4.1   1.86]
 [ 1.36  6.5   9.8 ]
 [ 2.16  3.88  3.7 ]
 [ 7.6   5.8   1.44]
 [ 1.86  4.1   4.6 ]
 [ 2.29  3.6   3.39]
 [ 2.44  3.95  2.9 ]
 [ 2.32  3.43  3.5 ]
 [ 1.23  8.5  15.  ]
 [ 3.95  3.52  2.14]
 [ 2.76  3.52  2.75]
 [ 3.86  3.85  2.05]
 [ 2.58  3.75  3.  ]
 [ 1.68  4.5   5.2 ]
 [ 4.    4.4   1.9 ]
 [ 2.6   4.33  2.64]
 [ 2.54  3.21  3.5 ]
 [ 2.79  3.69  2.7 ]
 [ 3.91  3.85  2.04]
 [ 2.54  3.87  2.9 ]
 [ 2.38  3.17  3.95]
 [ 1.72  3.7   6.75]
 [ 1.36  5.7  12.  ]
 [ 1.64  4.2   7.  ]
 [ 8.4   5.4   1.45]
 [ 1.76  4.16  5.2 ]
 [ 2.06  3.8   4.1 ]
 [ 1.87  4.    4.6 ]
 [ 1.73  4.3   5.5 ]
 [ 4.    3.75  2.08]
 [ 4.    4.3   1.93]
 [ 1.67  4.5   5.5 ]
 [ 2.9   3.55  2.68]
 [ 1.71  3.98  6.2 ]
 [ 3.02  3.62  2.57]
 [ 2.24  3.47  3.71]
 [ 1.56  4.6   6.8 ]
 [ 1.88  4.33  4.2 ]
 [ 1.88  4.3   4.1 ]
 [ 2.12  3.4   3.95]
 [ 1.89  4.2   4.2 ]
 [ 3.5   3.1   2.56]
 [ 1.65  4.3   6.5 ]
 [ 2.

In [202]:
result = resolve_fik(o, r, player_utility_kelly_criteria)
print(np.sum(result))
print(f'utility = {-player_utility_kelly_criteria(result, o, r)}')
print(f'expected_value = {player_gain_expected_value(result, o, r)}')
#print(f'variance = {player_gain_variance(result, o, r)}')
print(f'ecart_type = {np.sqrt(player_gain_variance(result, o, r))}')

1.0000000000000124
utility = 0.29884198558685554
expected_value = 0.47548517167722865
ecart_type = 0.35665140360837305


In [206]:
result[result < 1e-10] = 0
result

array([[0.        , 0.00592716, 0.02064907],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.01215391],
       [0.03665344, 0.        , 0.        ],
       [0.        , 0.02277339, 0.02106567],
       [0.        , 0.        , 0.01934714],
       [0.        , 0.        , 0.        ],
       [0.        , 0.0258554 , 0.02600968],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.01039033],
       [0.        , 0.        , 0.00022019],
       [0.        , 0.0227171 , 0.01606487],
       [0.        , 0.        , 0.01149344],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.04724042],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.04358477],
       [0.        , 0.        , 0.00396927],
       [0.        , 0.        , 0.00266422],
       [0.

In [208]:
df_models_results_joined[['f_home_kelly', 'f_draw_kelly', 'f_away_kelly']] = result
df_models_results_joined

,datetime_inference,model,game_id,game,date_match,time_match,home_team,away_team,prob_home_win,prob_draw,...,odds_away,bookmaker_home,bookmaker_draw,bookmaker_away,sport_key,sport_title,commence_time,f_home_kelly,f_draw_kelly,f_away_kelly
0,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Brentford-Southampton,2024-08-31,15:00:00,Brentford,Southampton,0.416884,0.281105,...,4.60,Matchbook,Suprabets,William Hill,soccer_epl,EPL,2024-08-31 14:00:00,0.000000,0.005927,0.020649
1,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Espanyol-Rayo Vallecano,2024-08-31,19:15:00,Espanyol,Rayo Vallecano,0.367470,0.307110,...,3.10,Betfair,1xBet,Tipico,soccer_spain_la_liga,La Liga - Spain,2024-08-31 17:15:00,0.000000,0.000000,0.000000
2,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Barcelona-Valladolid,2024-08-31,17:00:00,Barcelona,Valladolid,0.736859,0.183347,...,11.00,1xBet,Tipico,Nordic Bet,soccer_spain_la_liga,La Liga - Spain,2024-08-31 15:00:00,0.000000,0.000000,0.000000
3,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Athletic Club-Atlético Madrid,2024-08-31,19:00:00,Athletic Club,Atlético Madrid,0.252413,0.252999,...,2.57,Suprabets,1xBet,Suprabets,soccer_spain_la_liga,La Liga - Spain,2024-08-31 17:00:00,0.000000,0.000000,0.012154
4,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Leicester City-Aston Villa,2024-08-31,15:00:00,Leicester City,Aston Villa,0.335470,0.260304,...,1.86,Suprabets,Suprabets,Matchbook,soccer_epl,EPL,2024-08-31 14:00:00,0.036653,0.000000,0.000000
5,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Arsenal-Brighton,2024-08-31,12:30:00,Arsenal,Brighton,0.605428,0.223689,...,9.80,Unibet,Suprabets,Matchbook,soccer_epl,EPL,2024-08-31 11:30:00,0.000000,0.022773,0.021066
6,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Nott'ham Forest-Wolves,2024-08-31,15:00:00,Nott'ham Forest,Wolves,0.343544,0.291516,...,3.70,Matchbook,Suprabets,Matchbook,soccer_epl,EPL,2024-08-31 14:00:00,0.000000,0.000000,0.019347
7,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 West Ham-Manchester City,2024-08-31,17:30:00,West Ham,Manchester City,0.126607,0.176019,...,1.44,Matchbook,Suprabets,William Hill,soccer_epl,EPL,2024-08-31 16:30:00,0.000000,0.000000,0.000000
8,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Brest-Saint-Étienne,2024-08-31,17:00:00,Brest,Saint-Étienne,0.380743,0.323058,...,4.60,1xBet,Suprabets,Betfair,soccer_france_ligue_one,Ligue 1 - France,2024-08-31 15:00:00,0.000000,0.025855,0.026010
9,2024-08-27 06:32:52.650453,RSF_PR_LR,None,2024-08-31 Montpellier-Nantes,2024-08-31,19:00:00,Montpellier,Nantes,0.371483,0.319295,...,3.39,1xBet,Suprabets,Suprabets,soccer_france_ligue_one,Ligue 1 - France,2024-08-31 17:00:00,0.000000,0.000000,0.000000


In [209]:
df_models_results_joined.columns

Index(['datetime_inference', 'model', 'game_id', 'game', 'date_match',
       'time_match', 'home_team', 'away_team', 'prob_home_win', 'prob_draw',
       'prob_away_win', 'prob_home_team_score', 'prob_away_team_score',
       'match_id', 'odds_home', 'odds_draw', 'odds_away', 'bookmaker_home',
       'bookmaker_draw', 'bookmaker_away', 'sport_key', 'sport_title',
       'commence_time', 'f_home_kelly', 'f_draw_kelly', 'f_away_kelly'],
      dtype='object')

In [218]:
df_models_results_joined['datetime_optim'] = datetime.datetime.now()
df_models_results_joined_cols = df_models_results_joined[[
        'match_id', 'sport_key', 'game', 'date_match', 'time_match', 'home_team', 'away_team', 
        'model','datetime_inference', 'prob_home_win', 'prob_draw', 'prob_away_win',
        'odds_home', 'odds_draw', 'odds_away', 
        'bookmaker_home', 'bookmaker_draw', 'bookmaker_away', 'f_home_kelly', 'f_draw_kelly', 'f_away_kelly', 'datetime_optim']]

In [220]:
DB_TN_OPTIM_RESULTS = 'optim_results'

with engine.begin() as connection:
    df_models_results_joined_cols.to_sql(DB_TN_OPTIM_RESULTS, connection, if_exists='append', index=False)

In [242]:
def test(a, b, c, *args, **kwargs):
    print(a)
    print(b)
    print(c)
    print(args)
    print(kwargs)

In [243]:
test(*[1, 2, 3], [1, 2, 3], x="e", y={"a": 10, "b": 20})

1
2
3
([1, 2, 3],)
{'x': 'e', 'y': {'a': 10, 'b': 20}}
